# Eat Safe, Love

## Notebook Set Up

In [23]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [24]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [25]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [26]:
# review the collections in our database
collections = db.list_collection_names()
print(collections)

['establishments']


In [27]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [28]:
from pprint import pprint
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
hygiene_20_count = establishments.count_documents(query)
print(hygiene_20_count)
# Display the first document in the results using pprint
first_doc = establishments.find_one(query)
pprint(first_doc)

0
None


In [29]:
import pandas as pd

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
print(f"DataFrame rows: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))


DataFrame rows: 0
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [30]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex': 'London'}, 'RatingValue': {'$gte':4}}

# Use count_documents to display the number of documents in the result
establishments.count_documents(query)

# Display the first document in the results using pprint
pprint(establishments.find_one())


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RatingValue': None,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65dd0d6d65fc14f592a4f78d'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [31]:
# Convert the result to a Pandas DataFrame
london_high_rating_df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
print(len(london_high_rating_df))

# Display the first 10 rows of the DataFrame
london_high_rating_df.head(10)


0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [32]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query = {'RatingValue':5, 'geocode.latitude': {'$gte':latitude - degree_search, '$lte':latitude + degree_search},
         'geocode.longitude': {'$gte':longitude - degree_search, '$lte':longitude + degree_search}}
sort =  [('scores.Hygiene',1)]

result = establishments.find(query).sort(sort).limit(5)

results_df = pd.DataFrame(result)

# Display the number of rows and the first 10 rows in the DataFrame
num_rows = len(df)
print(f"Number of rows: {num_rows}")


Number of rows: 0


In [33]:
# Convert result to Pandas DataFrame
top_rated_df = pd.DataFrame(result)
top_rated_df

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [34]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match' : {'scores.Hygiene' : 0}}
# 2. Groups the matches by Local Authority
group_query = {'$group' : {'_id' : '$LocalAuthorityName', 'count':{'$sum':1}}}
# 3. Sorts the matches from highest to lowest
sort_values = {'$sort' : {'count' : -1}}
# Print the number of documents in the result
pipeline = [match_query, group_query, sort_values]
result = list(establishments.aggregate(pipeline))
print(len(result))
# Print the first 10 results
pprint(result[0:10])


0
[]


In [35]:
# Convert the result to a Pandas DataFrame
authority_df = pd.DataFrame(result)

# Display the number of rows in the DataFrame
print(len(authority_df))

# Display the first 10 rows of the DataFrame
authority_df.head(10)


0


""
